In [172]:
### ValDXer testing
import os


import pickle


In [173]:
import subprocess


In [174]:
MD_BPTI_dir = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3"


In [175]:
BPTI_dir = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI"
expt_dir = os.path.join(BPTI_dir, "BPTI_expt_data")

os.listdir(expt_dir)

segs_name = "BPTI_residue_segs.txt"
segs_path = os.path.join(expt_dir, segs_name)

hdx_name = "BPTI_expt_dfracs.dat"
hdx_path = os.path.join(expt_dir, hdx_name)
print(hdx_path)

rates_name = "BPTI_Intrinsic_rates.dat"
rates_path = os.path.join(expt_dir, rates_name)


/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat


In [183]:
sim_name = 'BPTI_MD'

sim_dir = os.path.join(MD_BPTI_dir)
print(sim_dir)

print(os.listdir(sim_dir))

md_reps = 5

rep_dirs = ["R_"+str(i+1) for i in range(md_reps)]

top_name = "APO_amber99_5PTI_1-nojump.pdb"

top_path = os.path.join(sim_dir, rep_dirs[0], top_name)

traj_name = "APO_amber99_5PTI_6-nojump.xtc"

# APO_amber99_5PTI_cat_{rep}.xtc

# this is the 

xtal_name= "5PTI_APO_amber99_H_solv_ions.gro"
xtal_path = os.path.join(sim_dir, rep_dirs[0], xtal_name)

traj_paths = [
    os.path.join(sim_dir, rep, f"APO_amber99_5PTI_cat_{idx+1}.xtc") for idx, rep in enumerate(rep_dirs)
]



print(top_path)
print(traj_paths)




/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3
['R_5', 'R_2', 'R_3', 'R_4', '5PTI_APO_amber99_H_solv_ions.gro', 'R_1']
/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_1/APO_amber99_5PTI_1-nojump.pdb
['/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_1/APO_amber99_5PTI_cat_1.xtc', '/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_2/APO_amber99_5PTI_cat_2.xtc', '/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_3/APO_amber99_5PTI_cat_3.xtc', '/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_4/APO_amber99_5PTI_cat_4.xtc', '/Users/alexi/Library/Clo

In [184]:
import MDAnalysis as mda
import MDAnalysis
import numpy as np
import pandas as pd


In [185]:
# load xtal
u = mda.Universe(top_path)
print(len(u.residues))
# select first frame
# u = u.trajectory[0]



print(type(u))   
# print residues


58
<class 'MDAnalysis.core.universe.Universe'>


In [216]:
def calc_HDX(structure: mda.Universe)->float:
    """This functions calculates the HDX-MS free energy using the Best Vendruscolo method
    For each residue this counts up the number of heavy atoms from the amide nitogren and 
    hydrogen bonds made with the amide hydrogen. The number of contacts are multiplied by
    their beta weights and then summed up to give the free energy of the protein.    

    """
    B_C, B_H = 0.35, 2.0
    cut_C, cut_H = 6.5, 2.4 # angstroms as in MDA

    n_C, n_H = 0, 0

    for res in structure.residues:
        resid = res.resid
        print("Resi: ", resid)

        amide_N = res.atoms.select_atoms("name N")
        amide_H = res.atoms.select_atoms("name H or name H1 or name H2 or name H3")
        # print(amide_N.atoms)
        amide_N_pos_string = " ".join([str(i) for i in amide_N.positions[0]])
        # print(amide_N_pos_string)

        # find heavy atoms within cut-off of amide nitrogen in universe
        heavy_atom_selection = f"point {amide_N_pos_string} {cut_C} and not name H* and not (name N and resid {resid})"
        heavy_atoms = structure.select_atoms(heavy_atom_selection)
        # print(len(heavy_atoms))
        n_C += len(heavy_atoms)

        amide_H_positions = amide_H.positions
        # repeat for acceptor atoms for all amide hydrogens
        for hydrogen in amide_H_positions:
            hydrogen_pos_string = " ".join([str(i) for i in hydrogen])
            # print(hydrogen_pos_string)
            acceptor_atom_selection = f"point {hydrogen_pos_string} {cut_H} and (type O or type N or type S) and not (name N and resid {resid})"
            acceptor_atoms = structure.select_atoms(acceptor_atom_selection)
            # print(len(acceptor_atoms))
            n_H += len(acceptor_atoms)

    HDX_free_energy = B_C * n_C + B_H * n_H

    return HDX_free_energy

    
        
print(calc_HDX(u))

1029.85


In [217]:
top = mda.Universe(top_path).select_atoms("protein")
print(top.atoms)


<AtomGroup [<Atom 1: N of type N of resname ARG, resid 1 and segid SYSTEM and altLoc >, <Atom 2: H1 of type H of resname ARG, resid 1 and segid SYSTEM and altLoc >, <Atom 3: H2 of type H of resname ARG, resid 1 and segid SYSTEM and altLoc >, ..., <Atom 890: C of type C of resname ALA, resid 58 and segid SYSTEM and altLoc >, <Atom 891: OC1 of type O of resname ALA, resid 58 and segid SYSTEM and altLoc >, <Atom 892: OC2 of type O of resname ALA, resid 58 and segid SYSTEM and altLoc >]>


In [218]:
top = mda.Universe(top_path).select_atoms("protein")
# select protein

print(calc_HDX(top))


1029.85


In [233]:
trajs = mda.Universe(top_path, traj_paths)
# select protein
print(len(trajs.trajectory))
# calc HDX over trajectory
stride = 100

traj_HDX = []
for idx, ts in enumerate(trajs.trajectory):
    if idx % stride == 0:
        print(idx)
        traj_HDX.append(calc_HDX(trajs))
    
    # print(calc_HDX(trajs))
    # break

/Users/alexi/miniconda/envs/HDXER_ENV/lib/python3.11/site-packages/MDAnalysis/coordinates/XDR.py:240: UserWarning:

Reload offsets from trajectory
 ctime or size or n_atoms did not match



51135
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18

In [234]:
# plot HDX over trajectory using plotly
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(y=traj_HDX))
fig.show()

In [238]:
# plot HDX as histogram using plotly bins = 100
fig = go.Figure(data=[go.Histogram(x=traj_HDX, nbinsx=200)])
fig.show()



In [241]:
xtal = mda.Universe(xtal_path)

# select protein
xtal = xtal.select_atoms("protein")
# print(xtal.atoms)

xtal_HDX= calc_HDX(xtal)

In [244]:
# plot HDX over trajectory using plotly dotted line at xtal HDX
fig = go.Figure(data=go.Scatter(y=traj_HDX))
fig.add_trace(go.Scatter(y=[xtal_HDX]*len(traj_HDX), mode='lines', name='xtal'))
# add title
fig.update_layout(title='HDX FE over 500ns MD trajectory')
# add xaxis labels
fig.update_xaxes(title_text='Frame')
# add yaxis labels
fig.update_yaxes(title_text='HDX FE /kcal mol-1')

fig.show()


In [249]:
import numpy as np

def calc_HDX_by_res(structure: mda.Universe) -> np.ndarray:
    B_C, B_H = 0.35, 2.0
    cut_C, cut_H = 6.5, 2.4  # Angstroms

    n_C, n_H = [], []

    for res in structure.residues:
        resid = res.resid
        print("Resi: ", resid)

        amide_N = res.atoms.select_atoms("name N")
        amide_H = res.atoms.select_atoms("name H or name H1 or name H2 or name H3")

        amide_N_pos_string = " ".join([str(i) for i in amide_N.positions[0]])

        heavy_atom_selection = f"point {amide_N_pos_string} {cut_C} and not name H* and not (name N and resid {resid})"
        heavy_atoms = structure.select_atoms(heavy_atom_selection)
        n_C.append(len(heavy_atoms))
        
        amide_H_positions = amide_H.positions
        total = 0
        for hydrogen in amide_H_positions:
            hydrogen_pos_string = " ".join([str(i) for i in hydrogen])
            acceptor_atom_selection = f"point {hydrogen_pos_string} {cut_H} and (type O or type N or type S) and not (name N and resid {resid})"
            acceptor_atoms = structure.select_atoms(acceptor_atom_selection)

            total += len(acceptor_atoms)

        # Handle case where no hydrogens are found
        if len(amide_H_positions) > 0:
            n_H.append(total / len(amide_H_positions))
        else:
            n_H.append(0)  # Or set a default value or skip the calculation

    n_C = np.array(n_C)
    n_H = np.array(n_H)

    HDX_FE_C = B_C * n_C
    HDX_FE_H = B_H * n_H
    HDX_FE = HDX_FE_C + HDX_FE_H  # Sum for each residue

    return HDX_FE

# Example usage
# hdx_bfactors = calc_HDX_by_res(u)


In [254]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
### THIS CANT 
# Load the trajectory and topology
trajs = mda.Universe(top_path, traj_paths)

# Select the reference (e.g., the backbone of the first frame)
ref = trajs.select_atoms('backbone')  # You can change 'backbone' to your selection

# Initialize the AlignTraj class with your universe, the reference universe, and the selection of atoms to be aligned
alignment = align.AlignTraj(trajs, trajs, select='backbone', in_memory=True)  # Change 'backbone' to your selection if needed

# Run the alignment
alignment.run()

print(len(trajs.trajectory))

# Calculate HDX over trajectory
stride = 100
hdx_bfactor_pdb_name = "BPTI_HDX_bfactors.pdb"
hdx_bfactor_pdb_path = os.path.join(sim_dir, hdx_bfactor_pdb_name)
print(hdx_bfactor_pdb_path)

with mda.Writer(hdx_bfactor_pdb_path, multiframe=True) as W:
    for idx, ts in enumerate(trajs.trajectory):
        if idx % stride == 0:
            hdx_bfactors = calc_HDX_by_res(trajs)
            # Add bfactor to structure for each residue
            for res, bfactor in zip(trajs.residues, hdx_bfactors):
                for atom in res.atoms:
                    atom.tempfactor = bfactor
            W.write(trajs)


51135
/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/BPTI_HDX_bfactors.pdb
Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58
Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  2

/Users/alexi/miniconda/envs/HDXER_ENV/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning:

Found no information for attr: 'formalcharges' Using default value of '0'

/Users/alexi/miniconda/envs/HDXER_ENV/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1198: UserWarning:

Found missing chainIDs. Corresponding atoms will use value of 'X'



Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58
Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54